<a href="https://colab.research.google.com/github/bhadreshpsavani/UnderstandingNLP/blob/master/DistilbertPerformance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers
!pip install -q datasets

     |████████████████████████████████| 1.8MB 8.1MB/s 
     |████████████████████████████████| 3.2MB 39.1MB/s 
     |████████████████████████████████| 890kB 48.3MB/s 
     |████████████████████████████████| 184kB 7.5MB/s 
     |████████████████████████████████| 245kB 13.7MB/s 
     |████████████████████████████████| 112kB 16.6MB/s 
     |████████████████████████████████| 20.7MB 1.4MB/s 


In [2]:
from transformers import squad_convert_examples_to_features
from transformers.data.processors.squad import SquadV2Processor, SquadFeatures
from tqdm.notebook import tqdm
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from datasets import load_metric, load_dataset
device =  'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [7]:
def get_result(predictions, references, dataset='squad_v2'):
  squad_metric = load_metric(dataset)
  score = squad_metric.compute(predictions=predictions, references=references)
  return score

def get_validation_data(dataset='squad_v2'):
  datasets = load_dataset(dataset)
  valid_dataset = datasets['validation']
  return valid_dataset

def get_infernece(valid_dataset, model, tokenizer, device):
  predictions=[]
  references=[]
  model.to(device)
  for example in tqdm(valid_dataset):
    inputs = tokenizer(example['question'], example['context'], return_tensors="pt")
    inputs.to(device)
    output = model(**inputs)
    start_index = torch.argmax(output['start_logits'])
    end_index = torch.argmax(output['end_logits'])
    ans_ids = inputs['input_ids'][0][start_index :end_index+1]
    answer = tokenizer.decode(ans_ids)
    answer = tokenizer.clean_up_tokenization(answer).strip()
    answer = '' if answer=='<s>' else answer
    no_answer_probability = 1 if len(example['answers']['answer_start'])!=0 else 0
    """
    predictions = [{'prediction_text': '1976', 'id': '56e10a3be3433e1400422b22', 'no_answer_probability': 0.}]
    references = [{'answers': {'answer_start': [97], 'text': ['1976']}, 'id': '56e10a3be3433e1400422b22'}]
    """
    pred = {'prediction_text':answer, 'id': example['id'], 'no_answer_probability' : no_answer_probability}
    ref = {'answers': example['answers'] , 'id': example['id']}
    predictions.append(pred)
    references.append(ref)
  return predictions, references

In [8]:
def print_results(model, tokenizer, device, dataset_name='squad_v2'):
  validation_dataset = get_validation_data(dataset_name)
  predictions, references = get_infernece(validation_dataset, model, tokenizer, device)
  score = get_result(predictions, references, dataset_name)
  print(score)

In [9]:
distilbert_path='twmkn9/distilbert-base-uncased-squad2'
distilbert_tokenizer = AutoTokenizer.from_pretrained(distilbert_path)
distilbert_model = AutoModelForQuestionAnswering.from_pretrained(distilbert_path)
distilbert_model.to(device)

DistilBertForQuestionAnswering(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            

In [10]:
print_results(distilbert_model, distilbert_tokenizer, device)

Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/9cac55034b086140f0649ecb5c604d09d7da2f2f5b73a90caa2e2bcc1f5cac09)


Token indices sequence length is longer than the specified maximum sequence length for this model (595 > 512). Running this sequence through the model will result in indexing errors


RuntimeError: ignored